In [7]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
df = pd.read_csv('movies.csv')

In [11]:
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [13]:
df.shape

(4803, 24)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [19]:
df.describe()

,index,budget,id,popularity,revenue,runtime,vote_average,vote_count
count,4803.000000,4.803000e+03,4803.000000,4803.000000,4.803000e+03,4801.000000,4803.000000,4803.000000
mean,2401.000000,2.904504e+07,57165.484281,21.492301,8.226064e+07,106.875859,6.092172,690.217989
std,1386.651002,4.072239e+07,88694.614033,31.816650,1.628571e+08,22.611935,1.194612,1234.585891
min,0.000000,0.000000e+00,5.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,1200.500000,7.900000e+05,9014.500000,4.668070,0.000000e+00,94.000000,5.600000,54.000000
50%,2401.000000,1.500000e+07,14629.000000,12.921594,1.917000e+07,103.000000,6.200000,235.000000
75%,3601.500000,4.000000e+07,58610.500000,28.313505,9.291719e+07,118.000000,6.800000,737.000000
max,4802.000000,3.800000e+08,459488.000000,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


In [31]:
select_feature = ['genres','keywords','tagline','cast','director']
print(select_feature)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [35]:
for feature in select_feature:
    df[feature] = df[feature].fillna('')

In [39]:
combine_feature = df['genres'] + df['keywords'] + df['tagline'] + df['cast'] + df['director']
combine_feature

0       Action Adventure Fantasy Science Fictioncultur...
1       Adventure Fantasy Actionocean drug abuse exoti...
2       Action Adventure Crimespy based on novel secre...
3       Action Crime Drama Thrillerdc comics crime fig...
4       Action Adventure Science Fictionbased on novel...
                              ...                        
4798    Action Crime Thrillerunited states\u2013mexico...
4799    Comedy RomanceA newlywed couple's honeymoon is...
4800    Comedy Drama Romance TV Moviedate love at firs...
4801    A New Yorker in ShanghaiDaniel Henney Eliza Co...
4802    Documentaryobsession camcorder crush dream gir...
Length: 4803, dtype: object

In [45]:
vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(combine_feature)

In [49]:
print(feature_vectors)

  (0, 3583)	0.16941894714909375
  (0, 20729)	0.2738578969459924
  (0, 16614)	0.15696073877453268
  (0, 14550)	0.2236809363648068
  (0, 23267)	0.16128139780622516
  (0, 26547)	0.19638671147741732
  (0, 22464)	0.20207089568711745
  (0, 21487)	0.21550128478931546
  (0, 27540)	0.19771357974524176
  (0, 27182)	0.23480088356130552
  (0, 21521)	0.1562356665943368
  (0, 18558)	0.2612168335104853
  (0, 18164)	0.08690831799482265
  (0, 27161)	0.1261988657937151
  (0, 24197)	0.07518543993419265
  (0, 22778)	0.2738578969459924
  (0, 4676)	0.2452909770424961
  (0, 26334)	0.13059723071916296
  (0, 22916)	0.33668756406929184
  (0, 10193)	0.16532432420812987
  (0, 4456)	0.21799675215510664
  (0, 9304)	0.2738578969459924
  (0, 21836)	0.09966592997173944
  (0, 8936)	0.11806131645084653
  (0, 444)	0.09109355212252294
  :	:
  (4801, 5449)	0.31510541707008866
  (4801, 22170)	0.31510541707008866
  (4801, 27361)	0.31510541707008866
  (4801, 27513)	0.3005604008026175
  (4801, 8121)	0.2611505149930138
  (4801,

In [55]:
similarity = cosine_similarity(feature_vectors)
print(similarity)

[[1.         0.06865296 0.01492221 ... 0.         0.         0.        ]
 [0.06865296 1.         0.02799128 ... 0.01243107 0.         0.        ]
 [0.01492221 0.02799128 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.01243107 0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [57]:
similarity.shape

(4803, 4803)

In [101]:
movie_name = input('Enter your Favourite movie name: ')

Enter your Favourite movie name:  godzilla


In [103]:
list_of_all_titles = df['title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [105]:
find_close_match = difflib.get_close_matches(movie_name,list_of_all_titles)
print(find_close_match)

['Shin Godzilla', 'Godzilla 2000']


In [107]:
close_match = find_close_match[0]
print(close_match)

Shin Godzilla


In [109]:
index_of_the_movie = df[df.title == close_match]['index'].values[0]
print(index_of_the_movie)

97


In [111]:
similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, 0.02045997945835025), (1, 0.012163098581268472), (2, 0.012509007143438524), (3, 0.009815881685515778), (4, 0.020051116604251292), (5, 0.005318799926956214), (6, 0.007161489494723491), (7, 0.020585291782041843), (8, 0.0073375727472582385), (9, 0.012246491509343243), (10, 0.02363850165466306), (11, 0.011260362364204708), (12, 0.007096199784811182), (13, 0.011755642657501694), (14, 0.020811733955932905), (15, 0.0067383789571282765), (16, 0.012935099367944628), (17, 0.011939721783456383), (18, 0.01313989597800119), (19, 0.010642785446013698), (20, 0.011738910717314427), (21, 0.004624815951461544), (22, 0.006657202641084443), (23, 0.006002272328530694), (24, 0.010083627000802654), (25, 0.003858429629412579), (26, 0.021894795929489334), (27, 0.018507886743264416), (28, 0.019921576677458232), (29, 0.012855778370564216), (30, 0.01214170352957315), (31, 0.020343390526748592), (32, 0.0), (33, 0.020774205504136254), (34, 0.0), (35, 0.014448193646727028), (36, 0.05244834750922538), (37, 0.007

In [113]:
len(similarity_score)

4803

In [115]:
sorted_similar_movies = sorted(similarity_score,key=lambda x:x[1],reverse=True)
print(sorted_similar_movies)

[(97, 1.0), (51, 0.10983754681973985), (4344, 0.1037772350095389), (4401, 0.07006603024642605), (2841, 0.06607340789833677), (4669, 0.06536859986658333), (1238, 0.0618168615935336), (1093, 0.06025961206748569), (3774, 0.060179965593733814), (4741, 0.059491183666266156), (4284, 0.05921397261361604), (3002, 0.057642433349319816), (4728, 0.0575320696858183), (67, 0.057219123147562764), (2086, 0.05638347569789129), (165, 0.05609366791755891), (312, 0.054391460264793606), (1471, 0.05255489546740712), (36, 0.05244834750922538), (473, 0.05243773975840889), (1110, 0.052153908666324515), (2304, 0.051163171392205886), (3941, 0.05057666749050746), (977, 0.05029032046273707), (3155, 0.0501420619285869), (661, 0.04970465506061225), (48, 0.04873208038527326), (2521, 0.0483128350676891), (1826, 0.048195569483807384), (189, 0.047662810630437084), (784, 0.046957006511607224), (4542, 0.0467702587297539), (3296, 0.04516403927612779), (341, 0.04502756419894298), (2751, 0.044626359926024144), (2237, 0.0441

In [129]:
movie_name = input('Enter the name of the movie :')
list_of_all_titles = df['title'].tolist()
find_close_match = difflib.get_close_matches(movie_name,list_of_all_titles)
close_match = find_close_match[0]
index_of_the_movie = df[df.title == close_match]['index'].values[0]
similarity_score = list(enumerate(similarity[index_of_the_movie]))
sorted_similar_movies = sorted(similarity_score,key=lambda x:x[1],reverse=True)
print('Your suggested Movies are : \n')

i = 1

for movie in sorted_similar_movies:
    index = movie[0]
    title_from_index = df[df.index==index]['title'].values[0]
    if(i<31):
        print(i, '.', title_from_index)
        i+=1

Enter the name of the movie : krish


Your suggested Movies are : 

1 . Krrish
2 . Kites
3 . My Beautiful Laundrette
4 . Fiza
5 . Indiana Jones and the Temple of Doom
6 . The Helix... Loaded
7 . Monsoon Wedding
8 . Broken Horses
9 . Namastey London
10 . Jab Tak Hai Jaan
11 . Neal 'n' Nikki
12 . Veer-Zaara
13 . Gandhi, My Father
14 . Red Planet
15 . The Tuxedo
16 . Hollow Man
17 . American Hero
18 . Conquest of the Planet of the Apes
19 . The Last Days on Mars
20 . Timecop
21 . Astro Boy
22 . Wing Commander
23 . Echo Dr.
24 . G.I. Joe: Retaliation
25 . Journey to Saturn
26 . Superman Returns
27 . Zombie Hunter
28 . Paycheck
29 . Hulk
30 . Heavy Metal
